# Домашня робота №3
Будемо використовувати *Збірник задач з теорії ймовірностей та математичної статистики: навч. посібник / В.В. Голомозий, М.В. Карташов, К.В. Ральченко. – К.: Видавничо-поліграфічний центр «Київський університет», 2015. – 366 с.*
Електронну версію збірника можна знайти [тут](http://probability.univ.kiev.ua/userfiles/kmv/gkr-problems.pdf).

In [6]:
import itertools
import numpy as np
import pandas as pd
from math import gcd, factorial, log
from collections import Counter

## 1 Задача 1.3.12
Знайти ймовiрнiсть того, що серед трьох цифр, кожна з яких вибрана навмання, буде лише 1, 2, 3 рiзних.

Обчисліть теоретичну (повним перебором) та еміричну (симулюванням $100000$ експериментів) імовірності.

In [116]:
def theoretical(condition):
    omega = 0
    fav = 0
    
    for el_event in itertools.product(range(10), repeat=3):
        fav += condition(el_event)
        omega += 1
    return fav / omega

def empirical(condition, simulations):
    fav = 0
    for i in range(simulations):
        el_event = np.random.choice(range(10), size=3, replace=True)
        fav += condition(el_event)
    return fav / simulations

simulations = 100_000
for k in range(1, 4):
    print("=" * 25)
    print(f"Theoretical probability of the event: P(k == {k}) = {theoretical(lambda event: len(set(event)) == k)}")
    print(f"Empirical probability of the event: P(k == {k}) = {empirical(lambda event: len(set(event)) == k, simulations)}")


Theoretical probability of the event: P(k == 1) = 0.01
Empirical probability of the event: P(k == 1) = 0.01021
Theoretical probability of the event: P(k == 2) = 0.27
Empirical probability of the event: P(k == 2) = 0.27086
Theoretical probability of the event: P(k == 3) = 0.72
Empirical probability of the event: P(k == 3) = 0.72256


# 2 Задача 1.3.14
З послiдовностi чисел $1, 2, . . . , n$ вибирають навмання $k$ рiзних чисел.
Яка ймовiрнiсть того, що:
1. кожне з вибраних чисел кратне даному числу $p$;
2. кожне з вибраних чисел кратне хоча б одному з двох взаємно простих чисел $p$ i $q$;
3. серед вибраних чисел є хоча б одне кратне $p$?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k, p, q$.
Виведіть результат для
- $n = 25, k = 5, p = 3, q = 4$;
- $n = 25, k = 10, p = 3, q = 4$.

In [63]:
simulations = 100_000

In [106]:
# event_1
def theoretical(n: int, k: int, p: int):
    omega = 0
    fav = 0
    value = factorial(k) # needed by classical definition of probability 
    for el_event in itertools.combinations(range(1, n+1), k):
        omega += value
        flag = 1
        for number in el_event:
            if number % p != 0:
                flag = 0
                break
        if flag:
            fav += value
    return fav / omega

def empirical(n: int, k: int, p: int, simulations: int):
    fav = 0
    for _ in range(simulations):
        flag = 1
        el_event = np.random.choice(range(1, n+1), size=k, replace=False)
        for number in el_event:
            if number % p != 0:
                flag = 0
                break
        if flag:
            fav += 1
    return fav / simulations       

th = [theoretical(25, 5, 3), theoretical(25, 10, 3)]
emp = [empirical(25, 5, 3, simulations), empirical(25, 10, 3, simulations)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=25, k=5, p=3)", "(n=25, k=10, p=3)"])
df

,"(n=25, k=5, p=3)","(n=25, k=10, p=3)"
theoretical,0.001054,0.0
empirical,0.001190,0.0


In [109]:
# event_2
def theoretical(n: int, k: int, p: int, q: int):
    if gcd(p, q) != 1:
        return 0
    omega = 0
    fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(range(1, n+1), k):
        omega += value
        flag = 1
        for number in el_event:
            if number % p != 0 and number % q != 0:
                flag = 0
                break
        if flag:
            fav += value
    return fav / omega

def empirical(n: int, k: int, p: int, q: int, simulations: int):
    if gcd(p, q) != 1:
        return 0
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(range(1, n+1), size=k, replace=False)
        flag = 1
        for number in el_event:
            if number % p != 0 and number % q != 0:
                flag = 0
                break
        if flag:
            fav += 1
    return fav / simulations

th = [theoretical(25, 5, 3, 4), theoretical(25, 10, 3, 4)]
emp = [empirical(25, 5, 3, 4, simulations), empirical(25, 10, 3, 4, simulations)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=25, k=5, p=3, q=4)", "(n=25, k=10, p=3, q=4)"])
df

,"(n=25, k=5, p=3, q=4)","(n=25, k=10, p=3, q=4)"
theoretical,0.014907,0.00002
empirical,0.014570,0.00003


In [118]:
# event_3
def theoretical(n: int, k: int, p: int):
    omega = 0
    fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(range(1, n+1), k):
        omega += value
        for number in el_event:
            if number % p == 0:
                fav += value
                break
    return fav / omega

def empirical(n: int, k: int, p: int, simulations: int):
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(range(1, n+1), size=k, replace=False)
        for number in el_event:
            if number % p == 0:
                fav += 1
                break
    return fav / simulations

th = [theoretical(25, 5, 3), theoretical(25, 10, 3)]
emp = [empirical(25, 5, 3, simulations), empirical(25, 10, 3, simulations)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=25, k=5, p=3)", "(n=25, k=10, p=3)"])
df

,"(n=25, k=5, p=3)","(n=25, k=10, p=3)"
theoretical,0.883531,0.99405
empirical,0.884140,0.99413


# 3 Задача 1.4.6
В коморi знаходяться $n$ пар черевикiв.
З них випадковим чином вибираються $2k$ черевикiв.
Яка ймовiрнiсть того, що серед вибраних черевикiв:
1. вiдсутнi парнi;
2. є рiвно одна комплектна пара;
3. є двi комплектнi пари?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k$.
Виведіть результат для
- $n = 8, k = 4$;
- $n = 13, k = 5$.

In [84]:
# suppose the boots are different
# event 1
def gen_boots(n: int):
    aux = ["left", "right"]
    number = [el for el in range(1, n + 1)]
    boots = itertools.product(number, aux)
    return boots
    
def theoretical(n: int, k: int):
    boots = gen_boots(n)
    omega = 0
    fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(boots, 2*k):
        numbers = [boot[0] for boot in el_event]
        fav += value * (max(Counter(numbers).values()) == 1)
        omega += value
    return fav / omega

def empirical(n: int, k: int, simulations: int):
    boots = [pair[0] for pair in gen_boots(n)]
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(boots, size=2*k, replace=False)
        fav += max(Counter(el_event).values()) == 1
    return fav / simulations

th = [theoretical(8, 4), theoretical(13, 5)]
emp = [empirical(8, 4, simulations), empirical(13, 5, simulations)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.019891,0.055135
empirical,0.021020,0.054740


In [101]:
# suppose the boots are different
# event 2
def theoretical(n: int, k: int, param: int, condition):
    boots = gen_boots(n)
    omega = 0
    fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(boots, 2*k):
        numbers = [boot[0] for boot in el_event]
        got = list(Counter(numbers).values()).count(2)
        if condition(got, param):
            fav += value
        omega += value
    return fav / omega

def empirical(n: int, k: int, simulations: int, param: int, condition):
    boots = [pair[0] for pair in gen_boots(n)]
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(boots, size=2*k, replace=False)
        got = list(Counter(el_event).values()).count(2)
        if condition(got, param):
            fav += 1
    return fav / simulations

c = lambda got, param: got == param
th = [theoretical(8, 4, 1, c), theoretical(13, 5, 1, c)]
emp = [empirical(8, 4, simulations, 1, c), empirical(13, 5, simulations, 1, c)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.278477,0.310136
empirical,0.278910,0.311060


In [103]:
# event 3
# we count the probability of the event = "There are at least 2 pairs."
c = lambda got, param: got >= param
th = [theoretical(8, 4, 2, c), theoretical(13, 5, 2, c)]
emp = [empirical(8, 4, simulations, 2, c), empirical(13, 5, simulations, 2, c)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.701632,0.634729
empirical,0.701720,0.635520


In [124]:
# suppose all left boots are same and all right boots are the same
# event 1
def gen_boots(n: int):
    boots = ["left" for i in range(n)] + ["right" for i in range(n)]
    return boots
    
def theoretical(n: int, k: int):
    boots = gen_boots(n)
    omega = fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(boots, 2*k):
        fav += value * (len(set(el_event)) == 1)
        omega += value
    return fav / omega

def empirical(n: int, k: int, simulations: int):
    boots = gen_boots(n)
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(boots, size=2*k, replace=False)
        fav += len(set(el_event)) == 1
    return fav / simulations

th = [theoretical(8, 4), theoretical(13, 5)]
emp = [empirical(8, 4, simulations), empirical(13, 5, simulations)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.000155,0.000108
empirical,0.000120,0.000140


In [126]:
# event 2  
def theoretical(n: int, k: int, param: int, condition):
    boots = gen_boots(n)
    omega = fav = 0
    value = factorial(k)
    for el_event in itertools.combinations(boots, 2*k):
        el_event = list(el_event)
        if condition(el_event.count("left"), el_event.count("right"), param):
            fav += value
        omega += value
    return fav / omega

def empirical(n: int, k: int, simulations: int, param: int, condition):
    boots = gen_boots(n)
    fav = 0
    for _ in range(simulations):
        el_event = np.random.choice(boots, size=2*k, replace=False)
        el_event = list(el_event)
        if condition(el_event.count("left"), el_event.count("right"), param):
            fav += 1
    return fav / simulations

c = lambda gl, gr, param: (gl == param and gr >= param) or (gl >= param and gr == param)
th = [theoretical(8, 4, 1, c), theoretical(13, 5, 1, c)]
emp = [empirical(8, 4, simulations, 1, c), empirical(13, 5, simulations, 1, c)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.009946,0.00350
empirical,0.010060,0.00329


In [127]:
# event 3
# we count the probability of the event = "There are at least 2 pairs."

c = lambda gl, gr, param: gl >= param and gr >= param
th = [theoretical(8, 4, 2, c), theoretical(13, 5, 2, c)]
emp = [empirical(8, 4, simulations, 2, c), empirical(13, 5, simulations, 2, c)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["(n=8, k=4)", "(n=13, k=5)"])
df

,"(n=8, k=4)","(n=13, k=5)"
theoretical,0.989899,0.996393
empirical,0.990180,0.996520


# 4 Задача 1.2.15
Нехай $\Omega = {1, 2, \ldots, 2n}$.
Всiм числам приписанi ймовiрностi, пропорцiйнi логарифмам цих чисел.
Знайти цi ймовiрностi.
Знайти ймовiрнiсть того, що в результатi експерименту з’явиться:
1. парне число;
2. непарне число.

Напишіть функцію для обрахунку еміричної (симулюванням $100000$ експериментів) імовірності в залежності від параметра $n$.
Виведіть результат для
- $n = 10$;
- $n = 25$.

In [104]:
def probabilities(n: int):
    probs = np.array([log(i) for i in range(1, 2*n+1)])
    probs = probs / sum(probs)
    return probs

In [130]:
def even_th(n: int):
    probs = probabilities(n)
    res = 0
    for i in range(1, 2*n, 2):
        res += probs[i]
    return res

def odd_th(n: int):
    probs = probabilities(n)
    res = 0
    for i in range(0, 2*n, 2):
        res += probs[i]
    return res

def empirical(n: int, simulations: int, param: int):
    fav = 0
    probs = probabilities(n)
    for i in range(simulations):
        event = np.random.choice(range(1, 2*n+1), p = probs)
        if event % 2 == param:
            fav += 1
    return fav / simulations
            
th = [even_th(10), odd_th(10), even_th(25), odd_th(25)]
s = 100_000
emp = [empirical(10, s, 0), empirical(10, s, 1), empirical(25,s, 0), empirical(25, s, 1)]
df = pd.DataFrame([th, emp], index = ["theoretical", "empirical"], columns = ["P(even), n = 10", "P(odd), n = 10", "P(even), n = 25", "P(odd), n = 25"])
df

,"P(even), n = 10","P(odd), n = 10","P(even), n = 25","P(odd), n = 25"
theoretical,0.520505,0.479495,0.507364,0.492636
empirical,0.518360,0.478330,0.504010,0.495610
